# QA of DSL Release v1.24
The purpose of this notebook is to review and test the new features in 1.23

Docs: https://docs.dimensions.ai/dsl/1.24.0-preview/


## Prerequisites

Please install the latest versions of these libraries to run this notebook. 

In [51]:
#
# load common libraries
import pandas as pd
from pandas.io.json import json_normalize

import time
import json
from tqdm.notebook import tqdm as progress

import plotly.express as px
from plotly.offline import plot 

import dimcli
from dimcli.shortcuts import *

dimcli.login(instance="integration")
dsl = dimcli.Dsl() 


DimCli v0.6.7 - Succesfully connected to <https://integration.dimensions.ai> (method: dsl.ini file)


---

## [DSL-363] 0 Returned as False

https://uberresearch.atlassian.net/browse/DSL-363

In [52]:
%dsldf search publications where field_citation_ratio = 0 return publications[field_citation_ratio]

Returned Publications: 20 (total = 13319376)


field_citation_ratio
0                    0.0
1                    0.0
2                    0.0
3                    0.0
4                    0.0
5                    0.0
6                    0.0
7                    0.0
8                    0.0
9                    0.0
10                   0.0
11                   0.0
12                   0.0
13                   0.0
14                   0.0
15                   0.0
16                   0.0
17                   0.0
18                   0.0
19                   0.0

## [DSL-401] Include version in response

https://uberresearch.atlassian.net/browse/DSL-401

In [53]:
%dsl search publications where field_citation_ratio = 0 return publications[field_citation_ratio]

Returned Publications: 20 (total = 13319376)


<dimcli.Dataset object #4715604496. Records: 20/13319376>

In [54]:
dsl_last_results._version

{'release': '1.24', 'version': '1.24.0'}

## [DSL-396] Describe command GRID

https://uberresearch.atlassian.net/browse/DSL-396

In [55]:
%dsl describe service grid

<dimcli.Dataset object #4715547792. Dict keys: 'release'>

## [DSL-400] Searching by data in datasets fails 

https://uberresearch.atlassian.net/browse/DSL-400

In [56]:
%dsl search datasets where date = "2020-01-02"

Returned Datasets: 20 (total = 58)


<dimcli.Dataset object #4715115216. Records: 20/58>

In [57]:
%dsl search datasets where date > "2020-01-02"

Returned Datasets: 20 (total = 53087)


<dimcli.Dataset object #4715581200. Records: 20/53087>

In [58]:
%dsl search datasets where date > "2020-01-02" and date <= "2020-01-03"

Returned Datasets: 20 (total = 277)


<dimcli.Dataset object #4715777360. Records: 20/277>

## [DSL-402] Add NUTS fields to GRID orgs 

https://uberresearch.atlassian.net/browse/DSL-402

**FILTERS TEST**

In [59]:
q = """
search organizations where {} is not empty return organizations[id+name+{}]
"""

fields = ["nuts_level1_code", "nuts_level1_name", "nuts_level2_code", "nuts_level2_name"]

for x in fields:
    print("====", x)
    d = dsl.query(q.format(x, x))
    print("\n".join([str(z) for z in d.organizations]))

==== nuts_level1_code
Returned Organizations: 20 (total = 32241)
{'id': 'grid.421091.f', 'nuts_level1_code': 'UKK', 'name': 'Engineering and Physical Sciences Research Council'}
{'id': 'grid.434257.3', 'nuts_level1_code': 'UKK', 'name': 'Economic and Social Research Council'}
{'id': 'grid.14467.30', 'nuts_level1_code': 'UKK', 'name': 'Science and Technology Facilities Council'}
{'id': 'grid.14105.31', 'nuts_level1_code': 'UKI', 'name': 'Medical Research Council'}
{'id': 'grid.8682.4', 'nuts_level1_code': 'UKK', 'name': 'Natural Environment Research Council'}
{'id': 'grid.418100.c', 'nuts_level1_code': 'UKK', 'name': 'Biotechnology and Biological Sciences Research Council'}
{'id': 'grid.426413.6', 'nuts_level1_code': 'UKK', 'name': 'Arts and Humanities Research Council'}
{'id': 'grid.438467.a', 'nuts_level1_code': 'UKI', 'name': 'Action on Hearing Loss'}
{'id': 'grid.507369.e', 'nuts_level1_code': 'UKF', 'name': 'Versus Arthritis'}
{'id': 'grid.25111.36', 'nuts_level1_code': 'AT1', 'nam

**FACETS TEST**

In [60]:
q = """
search organizations where {} is not empty return {}
"""

fields = ["nuts_level1_code", "nuts_level1_name", "nuts_level2_code", "nuts_level2_name"]

for x in fields:
    print("====", x)
    res = dsl.query(q.format(x, x))
    print("\n".join([str(z) for z in getattr(res, x)]))

==== nuts_level1_code
Returned Nuts_level1_code: 20
{'id': 'CZ0', 'count': 2747}
{'id': 'UKI', 'count': 1969}
{'id': 'FR1', 'count': 1611}
{'id': 'UKJ', 'count': 1049}
{'id': 'NL3', 'count': 950}
{'id': 'DEA', 'count': 928}
{'id': 'SE1', 'count': 847}
{'id': 'UKH', 'count': 705}
{'id': 'DE2', 'count': 694}
{'id': 'DE1', 'count': 680}
{'id': 'ITC', 'count': 619}
{'id': 'IE0', 'count': 616}
{'id': 'ITI', 'count': 609}
{'id': 'UKM', 'count': 587}
{'id': 'ES5', 'count': 583}
{'id': 'UKK', 'count': 575}
{'id': 'DK0', 'count': 570}
{'id': 'UKD', 'count': 548}
{'id': 'DE3', 'count': 511}
{'id': 'FR7', 'count': 506}
==== nuts_level1_name
Returned Nuts_level1_name: 20
{'id': 'ČESKÁ REPUBLIKA', 'count': 2747}
{'id': 'LONDON', 'count': 1969}
{'id': 'ÎLE DE FRANCE', 'count': 1611}
{'id': 'SOUTH EAST (ENGLAND)', 'count': 1049}
{'id': 'WEST-NEDERLAND', 'count': 950}
{'id': 'NORDRHEIN-WESTFALEN', 'count': 928}
{'id': 'ÖSTRA SVERIGE', 'count': 847}
{'id': 'EAST OF ENGLAND', 'count': 705}
{'id': 'BAYER

## [DSL-392] Publications research_orgs_name 

https://uberresearch.atlassian.net/browse/DSL-392

In [61]:
%dsldf search publications in authors for "\"Hook, D.W.\"" where research_org_names ~ "London"

Returned Publications: 20 (total = 29)


volume     type              id issue  \
0      14  article  pub.1115957159     5   
1       3  article  pub.1106289502   NaN   
2      95  article  pub.1085511076     5   
3      49  article  pub.1059174631    45   
4     113  article  pub.1038047274    23   
5     133  article  pub.1033519830     3   
6      45  article  pub.1032716976    44   
7     524  article  pub.1041921433   6‐7   
8      22  article  pub.1025632328     2   
9      44  article  pub.1028159934    37   
10    NaN  chapter  pub.1044195510   NaN   
11    325  article  pub.1050737187    11   
12     43  article  pub.1032115462    16   
13    104  article  pub.1036459050     6   
14     73  article  pub.1021542691     3   
15     42  article  pub.1006655528     2   
16     41  article  pub.1027415437    39   
17     41  article  pub.1012860743    35   
18     41  article  pub.1027141106    24   
19     41  article  pub.1020443337     1   

                                  author_affiliations  year  \
0   [[{'first_name': 'David', 'last_name': 'Zeitly...  2019   
1   [[{'first_name': 'Daniel W.', 'last_name': 'Ho...  2018   
2   [[{'first_name': 'Carl M.', 'last_name': 'Bend...  2017   
3   [[{'first_name': 'Carl M', 'last_name': 'Bende...  2016   
4   [[{'first_name': 'Carl M', 'last_name': 'Bende...  2014   
5   [[{'first_name': 'Carl M.', 'last_name': 'Bend...  2014   
6   [[{'first_name': 'Carl M', 'last_name': 'Bende...  2012   
7   [[{'first_name': 'D.W.', 'last_name': 'Hook', ...  2012   
8   [[{'first_name': 'Shadi', 'last_name': 'Gholiz...  2012   
9   [[{'first_name': 'Carl M', 'last_name': 'Bende...  2011   
10  [[{'first_name': 'Carl M.', 'last_name': 'Bend...  2011   
11  [[{'first_name': 'Carl M.', 'last_name': 'Bend...  2010   
12  [[{'first_name': 'Carl M', 'last_name': 'Bende...  2010   
13  [[{'first_name': 'Carl M.', 'last_name': 'Bend...  2010   
14  [[{'first_name': 'Carl M.', 'last_name': 'Bend...  2009   
15  [[{'first_name': 'Dorje C', 'last_name': 'Brod...  2009   
16  [[{'first_name': 'Carl M', 'last_name': 'Bende...  2008   
17  [[{'first_name': 'Carl M', 'last_name': 'Bende...  2008   
18  [[{'first_name': 'Carl M', 'last_name': 'Bende...  2008   
19  [[{'first_name': 'D W', 'last_name': 'Hook', '...  2008   

                                                title      pages  \
0                   Perception, prestige and PageRank   e0216783   
1   Dimensions: Building Context for Search and Ev...         23   
2   Behavior of eigenvalues in a region of broken ...     052113   
3   -symmetric interpretation of unstable effectiv...     45lt01   
4   Infinite Class of PT-Symmetric Theories from O...     231605   
5   Complex Classical Motion in Potentials with Po...    318-336   
6   Negative-energy $\mathcal {P}\mathcal {T}$-sym...     444003   
7       Non‐Hermitian potentials and real eigenvalues    106-107   
8   The Discursive Construction of the 1978–1979 I...    174-186   
9            Quantum tunneling as a classical anomaly     372001   
10                          Complex elliptic pendulum       1-18   
11           Probability density in the complex plane  2332-2362   
12  Classical particle in a complex elliptic poten...     165201   
13                   Complex Correspondence Principle     061601   
14             Chaotic systems in complex phase space        453   
15  Information geometry in vapour–liquid equilibrium     023001   
16  Conjecture on the analyticity of {\cal PT} -sy...     392005   
17  Quantum effects in classical systems having co...     352003   
18  Exact isospectral pairs of {\cal P}{\cal T} sy...     244005   
19                         Geometry of Quantum States     019001   

      journal.id                                      journal.title  
0   jour.1037553                                           PLoS ONE  
1   jour.1292498        Frontiers in Research Metrics and Analytics  
2   jour.1053349                                  Physical Review A  
3   jour.1084797  Journal of Physics A: Mathe

## [DSL-403] Boosting full text syntax

https://uberresearch.atlassian.net/browse/DSL-403

Can see the new docs on https://docs.dimensions.ai/dsl/1.24.0-preview/language.html#term-boosting

In [62]:
%dsldf search publications for "\"digital humanities\"^4 history ontology" where type="article" return publications[title+journal+year]

Returned Publications: 20 (total = 1387)


title  year    journal.id  \
0   Describing, comparing and analysing digital ur...  2020  jour.1151279   
1   Practical non-monotonic knowledge-base system ...  2020  jour.1121462   
2   “Just-in-time” generation of datasets by consi...  2020  jour.1041769   
3   Digital Cultural Heritage and Linked Data: Sem...  2020  jour.1140306   
4   ConVRgence (VRIC) Virtual Reality Internationa...  2020  jour.1051167   
5   Report of the ALCTS/LITA Linked Library Data I...  2020  jour.1125087   
6   Affective Encounters with Tidal Livelihoods: D...  2020  jour.1142147   
7   Listening to Indigenous Knowledge of the Land ...  2020  jour.1151291   
8   The future of travel writing: Interactivity, p...  2020  jour.1052651   
9   Continuous, not discrete: The mutual influence...  2020  jour.1138940   
10  The enslaved ontology: Peoples of the historic...  2020  jour.1033697   
11  The finger of God is here! Past, present, and ...  2020  jour.1024673   
12  A retrospective on Telos as a metamodeling lan...  2020  jour.1136300   
13  French Media Representations towards Sustainab...  2020  jour.1046556   
14              Paradoxes of curating colonial memory  2020  jour.1028231   
15  More than just a product: Strengthening litera...  2020  jour.1049941   
16  Stan i perspektywy badań geograficzno-historyc...  2020  jour.1019105   
17  Tracking the history and evolution of entities...  2020  jour.1135728   
18        Information professionals meet Arthur Prior  2020  jour.1120510   
19  Spatial Archaeology: Mapping the Ancient Past ...  2020  jour.1100760   

                                        journal.title  
0   Digital Applications in Archaeology and Cultur...  
1                 Information Processing & Management  
2                   Knowledge and Information Systems  
3                                     LIBER Quarterly  
4            International Journal of Virtual Reality  
5                        Technical Services Quarterly  
6                         Environmental Communication  
7                            Feminist Media Histories  
8                                  Tourism Management  
9   Convergence The International Journal of Resea...  
10                           Journal of Web Semantics  
11                                          Historian  
12                           Requirements Engineering  
13                                     Sustainability  
14                                   Archival Science  
15                                  Sociology Compass  
16    TSG - Tijdschrift voor gezondheidswetenschappen  
17         International Journal on Digital Libraries  
18   Journal of Librarianship and Information Science  
19  International Journal of Humanities and Arts C...

## [DSL-399] Add two new categorizations to the classify API

https://uberresearch.atlassian.net/browse/DSL-399

In [63]:
abstract = """
What does it mean for a student to come to an understanding of a philosophical standpoint and can the explosion of resources now available on the web support this process, or is it inclined instead to create more confusion? We believe that a possible answer to the problem of finding a means through the morass of information on the web to the philosophical insights it conceals lies in the process of narrative pathway generation. That is, the active linking of resources into a learning path that contextualizes them with respect to one another. This result can be achieved only if the content of the resources is indexed, not just their status as a text document, an image or a video. To this aim, we propose a formal conceptualization of the domain of philosophy, an ontology that would allow the categorization of resources according to a series of pre-agreed content descriptors. Within an e-learning scenario, a teacher could use a tool comprising such an ontology to annotate at various levels of granularity available philosophical materials, and let the students explore this semantic space in an unsupervised manner, according to pre-defined narrative pathways.
"""
title = "Semantic Technologies for ELearning: A case study in the philosophical domain: representing Wittgenstein's Tractatus Logico-Philosophicus"

In [64]:
q = """
classify(title="{}", abstract="{}", system="UOA")
"""
qq = q.format(title, abstract)
print(qq)
dsl.query(qq).as_dataframe()


classify(title="Semantic Technologies for ELearning: A case study in the philosophical domain: representing Wittgenstein's Tractatus Logico-Philosophicus", abstract="
What does it mean for a student to come to an understanding of a philosophical standpoint and can the explosion of resources now available on the web support this process, or is it inclined instead to create more confusion? We believe that a possible answer to the problem of finding a means through the morass of information on the web to the philosophical insights it conceals lies in the process of narrative pathway generation. That is, the active linking of resources into a learning path that contextualizes them with respect to one another. This result can be achieved only if the content of the resources is indexed, not just their status as a text document, an image or a video. To this aim, we propose a formal conceptualization of the domain of philosophy, an ontology that would allow the categorization of resources acc

id                                  name
0  30011  B11 Computer Science and Informatics

In [80]:
q = """
classify(title="{}", abstract="{}", system="SDG")
"""
qq = q.format(title, abstract)
print(qq)
dsl.query(qq).as_dataframe()


classify(title="Semantic Technologies for ELearning: A case study in the philosophical domain: representing Wittgenstein's Tractatus Logico-Philosophicus", abstract="
What does it mean for a student to come to an understanding of a philosophical standpoint and can the explosion of resources now available on the web support this process, or is it inclined instead to create more confusion? We believe that a possible answer to the problem of finding a means through the morass of information on the web to the philosophical insights it conceals lies in the process of narrative pathway generation. That is, the active linking of resources into a learning path that contextualizes them with respect to one another. This result can be achieved only if the content of the resources is indexed, not just their status as a text document, an image or a video. To this aim, we propose a formal conceptualization of the domain of philosophy, an ontology that would allow the categorization of resources acc

Empty DataFrame
Columns: []
Index: []

In [82]:
dsl.query("""
classify(title="Modeling of flow uniformity by installing inlet distributor within the inflow part of a pressurized module using computational fluid dynamics", abstract="Uniform flow distribution is a significant parameter for designing pressurized membrane modules because non-uniform flow distribution can cause serious local flux and fouling problems within a module. Thus, this study investigated the fluid behavior with regards to the evenness of water distribution using newly designed inlet distributors in the inflow part of a pressurized membrane module. From the results of velocity and pressure at the cross-sectional and outlet planes, we confirmed that a conventional membrane module with no distributor (non-distributor) had fluid that was concentrated at the central part. Case 1, which had a cross-shaped distributor, reduced the central concentration tendency, and Case 2, which had a round-shaped distributor, displayed a relatively uniform flow based on the velocity, pressure, flux, and standard deviation data. Here, the non-uniformity coefficient (N) and energy utilization (η) for Cases 1 and 2 showed a lower non-uniformity coefficient (0.030 and 0.017, respectively) than for the Non-distributor (0.039). The energy utilization of Cases 1 and 2 were higher (1.35e-0.5 and 1.46e-05) than the Non-distributor (1.64e-05). Overall, we confirmed that the inlet distributors led to increased evenness of flow distribution within an inflow part.", system="SDG")
""").as_dataframe()

id                           name
0  40007  7 Affordable and Clean Energy

## [DSL-398] LuceneParser: deal with apostrophe (bug)

https://uberresearch.atlassian.net/browse/DSL-398

In [66]:
%%dsldf

search publications in title_abstract_only for "huntington's disease"
    where year in [2016:2020]
    return research_orgs[id+types+name] limit 200

Returned Research_orgs: 200


id  count         types                              name
0    grid.83440.3b    213   [Education]         University College London
1    grid.17091.3e    130   [Education]    University of British Columbia
2      grid.5600.3    130   [Education]                Cardiff University
3    grid.10419.3d    129  [Healthcare]  Leiden University Medical Center
4    grid.214572.7    119   [Education]                University of Iowa
..             ...    ...           ...                               ...
195  grid.15444.30     11   [Education]                 Yonsei University
196    grid.1649.a     11  [Healthcare]   Sahlgrenska University Hospital
197  grid.16872.3a     11  [Healthcare]      VU University Medical Center
198  grid.22072.35     11   [Education]             University of Calgary
199  grid.223827.e     11   [Education]                University of Utah

[200 rows x 4 columns]

In [67]:
%%dsldf

search publications in title_abstract_only for "Giovanna D'Arco"
    return publications limit 10

Too Many Requests for the Server. Sleeping for 30 seconds and then retrying.
Returned Publications: 10 (total = 18)


volume     type              id issue  \
0     13  article  pub.1123722405     2   
1     62  article  pub.1090551325     1   
2    NaN  article  pub.1071130898    21   
3    NaN  chapter  pub.1012392714   NaN   
4      4  article  pub.1100984863     1   
5     60  article  pub.1009399440     3   
6    NaN  article  pub.1034827718    25   
7     86  article  pub.1059910469     2   
8     21  article  pub.1059955903     2   
9    NaN  chapter  pub.1100330910   NaN   

                                 author_affiliations  year  \
0  [[{'first_name': 'Meta Lenart', 'last_name': '...  2019   
1  [[{'first_name': 'Adél', 'last_name': 'Fekete'...  2017   
2  [[{'first_name': 'Clizia', 'last_name': 'Magon...  2016   
3  [[{'first_name': 'Anselm', 'last_name': 'Gerha...  2013   
4  [[{'first_name': '', 'last_name': 'Friba of RA...  2008   
5  [[{'first_name': 'Francesco', 'last_name': 'Iz...  2007   
6  [[{'first_name': 'Julie', 'last_name': 'DERAMO...  2007   
7  [[{'first_name': 'Alberto', 'last_name': 'Rizz...  2005   
8  [[{'first_name': 'G.', 'last_name': 'Martin', ...  2005   
9  [[{'first_name': 'Roger', 'last_name': 'Parker...  2002   

                                               title    pages    journal.id  \
0  Sulle tracce di Giovanna d’Arco tra mito e doc...  119-131  jour.1143410   
1  IV. Prayers of Female Protagonists in Giuseppe...  203-217  jour.1299915   
2            Giovanna d'Arco, una storia senza tempo    52-56  jour.1145232   
3                                    Giovanna d’Arco  370-374           NaN   
4  Life sketch of martyred Meena, founding leader...    21-34  jour.1276887   
5  Verdi, the Virgin, and the Censor: The Politic...  557-597  jour.1137058   
6  Jeanne d'Arc et ses voix, dans deux opéras, Ve...      NaN  jour.1041485   
7  Viganò’s ‘Giovanna d’Arco’ and Manzoni’s ‘Marc...  186-201  jour.1025189   
8  Verdi Onstage in the United States: Giovanna d...  242-263  jour.1051906   
9                                    Giovanna d’Arco      NaN           NaN   

                                   journal.title  
0                                Ars & Humanitas  
1       Studia Universitatis Babeş-Bolyai Musica  
2                              Historia Magistra  
3                                            NaN  
4                             Storia delle Donne  
5  Journal of the American Musicological Society  
6                                           Clio  
7                              Music and Letters  
8                            The Opera Quarterly  
9                                            NaN

## [DSL-362] Be more robust in dealing with quotes messed up by slack/google docs

https://uberresearch.atlassian.net/browse/DSL-362

In [68]:
q1 = """search publications for 
“\”2019-nCoV\” OR \“COVID-19\” OR \“SARS-CoV-2\” OR \“HCoV-2019\” OR \“hcov\” OR \“NCOVID-19\” OR \“severe acute respiratory syndrome coronavirus 2\” OR \“severe acute respiratory syndrome corona virus 2\” OR ((\“coronavirus\” OR \“corona virus\“) AND (Wuhan OR China OR novel))”
 return publications limit 1"""

In [69]:
dsl.query(q1)

Returned Publications: 1 (total = 9503)


<dimcli.Dataset object #4716160656. Records: 1/9503>

#### Can't reproduce the query from Bob though..

Prob just an issue of too many quotes - a smaller version of the query works (see below)

In [70]:
q2 = """search publications for 
(“adverse event” OR AE OR safety OR toxicity OR tolerability OR "Antibodies, Monoclonal, Humanized/adverse effects"[MeSH Terms] OR "Antineoplastic Agents/adverse effects"[MAJR] OR hypothyroidism OR hyperthyroidism OR rash OR dermatitis OR pruritus OR colitis OR diarrhea OR hepatitis OR hepatotoxicity OR Nausea OR arthritis OR arthralgia OR fatigue OR “Infusion reaction” OR “Infusion-related” OR hypophysitis OR pancreatitis OR “cardiac events” OR “MACE” OR “CV events” or “Cardiovascular events” OR “thrombotic events” OR hemorrhage OR myocarditis) AND (“renal cell carcinoma” OR "Carcinoma, Renal Cell"[MeSH]) AND ((TKI OR “tyrosine kinase inhibitor” OR "Protein Kinase Inhibitors" [Pharmacological Action] OR axitinib OR cabozantinib OR lenvatinib OR sunitinib) OR (“Antineoplastic Agents, Immunological”[Pharmacological Action] OR “programmed death ligand 1” OR PD-L1 OR avelumab OR bavencio OR “programmed cell death protein 1” OR PD-1 OR pembrolizumab OR keytruda OR “mammalian target of rapamycin” OR everolimus OR afinitor OR nivolumab OR opdivo OR “checkpoint inhibitor”)) AND (“first line” OR first-line OR front-line OR “front line” OR frontline OR naïve OR naïve OR untreated)
return publications limit 1 """
dsl.query(q2)

Returned Errors: 1
4 QuerySyntaxErrors found
2 LexerErrors found
  * [Line 2:1164] ('ïve OR naïve OR untreated)') token recognition error at: 'ï'
  * [Line 2:1173] ('ïve OR untreated)') token recognition error at: 'ï'
2 ParserErrors found
  * [Line 2:0] ('(') extraneous input '(' expecting {'text', NAME, STRING}
  * [Line 2:17] ('OR') mismatched input 'OR' expecting 'return'


<dimcli.Dataset object #4716021328. Errors: 1>

In [71]:
%%dsl 

search publications for 
"(\“adverse event\” OR AE OR safety OR toxicity OR tolerability OR \"Antibodies, Monoclonal, Humanizedadverse effects\")"

return publications limit 1

Returned Publications: 1 (total = 5633079)


<dimcli.Dataset object #4716021072. Records: 1/5633079>

## [DSL-309] Units of Assessment for Grants

https://uberresearch.atlassian.net/browse/DSL-309

get some stats

In [72]:
dslquery("search grants where category_uoa is not empty return grants").stats

Returned Grants: 20 (total = 3346986)


{'total_count': 3346986}

get an overview of how many docs per category

In [73]:
%dsldf search publications return category_uoa 

Returned Category_uoa: 20


id     count                                               name
0   30012  13039569                                    B12 Engineering
1   30001   7723142                              A01 Clinical Medicine
2   30003   5479395  A03 Allied Health Professions, Dentistry, Nurs...
3   30011   3501897               B11 Computer Science and Informatics
4   30008   2684717                                      B08 Chemistry
5   30005   2602474                            A05 Biological Sciences
6   30009   2514556                                        B09 Physics
7   30004   2372525        A04 Psychology, Psychiatry and Neuroscience
8   30010   2359287                          B10 Mathematical Sciences
9   30006   2274657       A06 Agriculture, Veterinary and Food Science
10  30017   2040397                C17 Business and Management Studies
11  30007   1626369       B07 Earth Systems and Environmental Sciences
12  30023    969936                                      C23 Education
13  30002    850443  A02 Public Health, Health Services and Primary...
14  30014    668462            C14 Geography and Environmental Studies
15  30028    469930                                        D28 History
16  30024    452134  C24 Sport and Exercise Sciences, Leisure and T...
17  30013    402851   C13 Architecture, Built Environment and Planning
18  30020    392142                  C20 Social Work and Social Policy
19  30018    383644                                            C18 Law

search docs with two different categories

In [74]:
%dsldf search grants where category_uoa.id="30004" and category_uoa.name!="D28 History"

Returned Grants: 20 (total = 147615)
WARNINGS [1]
Query is too long or complex. Please see https://docs.dimensions.ai/dsl/faq.html for more information. [code: 4]


funders title_language  \
0   [{'id': 'grid.425888.b', 'linkout': ['http://w...             en   
1   [{'id': 'grid.270680.b', 'linkout': ['http://e...             en   
2   [{'id': 'grid.270680.b', 'linkout': ['http://e...             en   
3   [{'id': 'grid.270680.b', 'linkout': ['http://e...             en   
4   [{'id': 'grid.270680.b', 'linkout': ['http://e...             en   
5   [{'id': 'grid.452896.4', 'linkout': ['http://e...             en   
6   [{'id': 'grid.270680.b', 'linkout': ['http://e...             en   
7   [{'id': 'grid.270680.b', 'linkout': ['http://e...             en   
8   [{'id': 'grid.270680.b', 'linkout': ['http://e...             en   
9   [{'id': 'grid.270680.b', 'linkout': ['http://e...             en   
10  [{'id': 'grid.452896.4', 'linkout': ['http://e...             en   
11  [{'id': 'grid.454063.0', 'linkout': ['http://w...             en   
12  [{'id': 'grid.434257.3', 'linkout': ['http://w...             en   
13  [{'id': 'grid.270680.b', 'linkout': ['http://e...             en   
14  [{'id': 'grid.270680.b', 'linkout': ['http://e...             en   
15  [{'id': 'grid.270680.b', 'linkout': ['http://e...             en   
16  [{'id': 'grid.270680.b', 'linkout': ['http://e...             en   
17  [{'id': 'grid.270680.b', 'linkout': ['http://e...             en   
18  [{'id': 'grid.270680.b', 'linkout': ['http://e...             en   
19  [{'id': 'grid.270680.b', 'linkout': ['http://e...             en   

                        funding_org_name    end_date  start_date  \
0      Swiss National Science Foundation  2021-08-31  2021-06-01   
1                    European Commission  2023-05-02  2021-05-03   
2                    European Commission  2023-01-02  2021-01-03   
3                    European Commission  2022-12-31  2021-01-01   
4                    European Commission  2022-11-30  2020-12-01   
5              European Research Council  2025-10-31  2020-11-01   
6                    European Commission  2022-10-11  2020-10-12   
7                    European Commission  2023-09-30  2020-10-01   
8                    European Commission  2023-03-31  2020-10-01   
9                    European Commission  2022-09-30  2020-10-01   
10             European Research Council  2025-09-30  2020-10-01   
11                    Wolfson Foundation  2025-10-01  2020-10-01   
12  Economic and Social Research Council  2024-09-29  2020-09-30   
13                   European Commission  2022-09-13  2020-09-14   
14                   European Commission  2022-08-31  2020-09-01   
15                   European Commission  2022-08-31  2020-09-01   
16                   European Commission  2022-08-31  2020-09-01   
17                   European Commission  2022-08-31  2020-09-01   
18                   European Commission  2022-08-31  2020-09-01   
19                   European Commission  2022-08-31  2020-09-01   

                                       original_title language  start_year  \
0                    Gender and Cancer Susceptibility       en        2021   
1   The Social INdividual’s Decisions: how are the...       en        2021   
2   Model-based biomarkers for the characterizatio...       en        2021   
3   Convergence of Information from Separate Brain...       en        2021   
4   Multi-layered integration of motivated actions...       en        2020   
5   Separating parallel threads of cognition to be...       en        2020   
6   Common mechanisms for decision-making and work...       en        2020   
7   Being ‘Mad’ in Byzantium. Toward a History of ...       en        2020   
8   Characterize corpus callosum-mediated local an...       en        2020   
9    DEciphering mechanisms of presynaptic reFINEment       en        2020   
10  Sleep and episodic memory consolidation:  ‘No-...       en        2020   
11    The Wolfson Centre for Adolescent Mental Health       en        2020   
12  Superior perceptual capacity in autism: invest...       en        2020   
13  Soluble 

In [75]:
%dsldf search grants where category_uoa.id="30004" return category_uoa

Returned Category_uoa: 20


id   count                                               name
0   30004  147615        A04 Psychology, Psychiatry and Neuroscience
1   30005    1449                            A05 Biological Sciences
2   30003     512  A03 Allied Health Professions, Dentistry, Nurs...
3   30001     328                              A01 Clinical Medicine
4   30011     327               B11 Computer Science and Informatics
5   30023     196                                      C23 Education
6   30026     120               D26 Modern Languages and Linguistics
7   30012      98                                    B12 Engineering
8   30017      75                C17 Business and Management Studies
9   30024      56  C24 Sport and Exercise Sciences, Leisure and T...
10  30020      42                  C20 Social Work and Social Policy
11  30002      34  A02 Public Health, Health Services and Primary...
12  30006      22       A06 Agriculture, Veterinary and Food Science
13  30033      17  D33 Music, Drama, Dance, Performing Arts, Film...
14  30030      13                                     D30 Philosophy
15  30018      10                                            C18 Law
16  30022       4           C22 Anthropology and Development Studies
17  30008       1                                      B08 Chemistry
18  30009       1                                        B09 Physics
19  30010       1                          B10 Mathematical Sciences

search with partial string matching

In [76]:
%dsldf search grants where category_uoa.name~"history" return category_uoa

Returned Category_uoa: 20


id  count                                               name
0   30028  28855                                        D28 History
1   30032  16550   D32 Art and Design: History, Practice and Theory
2   30012    105                                    B12 Engineering
3   30019     79             C19 Politics and International Studies
4   30011     65               B11 Computer Science and Informatics
5   30026     55               D26 Modern Languages and Linguistics
6   30023     53                                      C23 Education
7   30018     51                                            C18 Law
8   30027     45                D27 English Language and Literature
9   30013     44   C13 Architecture, Built Environment and Planning
10  30017     43                C17 Business and Management Studies
11  30034     36  D34 Communication, Cultural and Media Studies,...
12  30033     27  D33 Music, Drama, Dance, Performing Arts, Film...
13  30025     25                                   D25 Area Studies
14  30015     22                                    C15 Archaeology
15  30031     11                 D31 Theology and Religious Studies
16  30024      9  C24 Sport and Exercise Sciences, Leisure and T...
17  30014      8            C14 Geography and Environmental Studies
18  30020      8                  C20 Social Work and Social Policy
19  30029      7                                       D29 Classics

In [77]:
%dsldf search grants where category_uoa.name~"history" return researchers limit 10

Returned Researchers: 10


id  count     first_name    last_name  \
0  ur.015202435305.87     46         Grants         Team   
1  ur.010320437431.61     22            E I  Kolchinskii   
2   ur.01051707425.47     22         Ulrich      Herbert   
3  ur.016300720353.30     20  Rosalie Burns        Davis   
4  ur.010205156151.22     16            Kei         Arai   
5  ur.016627753011.35     16         Seishi       Namiki   
6   ur.01111215176.48     15     Virginia S     Berridge   
7  ur.015573721121.33     15          Keizo        Asaji   
8  ur.016367164225.36     15          Osamu        Saito   
9  ur.010751216401.00     14        Shigeru        Akita   

                               research_orgs               orcid_id  
0                                        NaN                    NaN  
1                                        NaN                    NaN  
2                              [grid.5963.9]                    NaN  
3                                        NaN                    NaN  
4                                        NaN                    NaN  
5                                        NaN                    NaN  
6  [grid.4563.4, grid.8991.9, grid.83440.3b]  [0000-0001-8011-8348]  
7                            [grid.444507.6]                    NaN  
8                                        NaN                    NaN  
9                                        NaN                    NaN

## [DSL-406] Deprecate Patents.assignee_state_names and clarify docs for assignee fields 

https://uberresearch.atlassian.net/browse/DSL-406

In [78]:
%dsl search patents where id="EP-3225971-A1" return patents[assignee_names+assignees+assignee_countries]

Returned Patents: 1 (total = 1)


<dimcli.Dataset object #4716236496. Records: 1/1>

In [79]:
%dsl search patents where id="EP-3225971-A1" return patents[assignee_names+assignee_state_names]

Returned Patents: 1 (total = 1)
WARNINGS [1]
Field 'assignee_state_names' is deprecated. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details


<dimcli.Dataset object #4716233552. Records: 1/1>